# Data Load

In [2]:
import pandas as pd
import os

In [3]:
os.chdir('/Users/aidenguerin/Desktop/projects/snow-vis/fit3179-vis02')

In [4]:
# Load csv
pass_info = pd.read_csv('web-scraping/data/pass_info.csv')
resort_elevation = pd.read_csv('web-scraping/data/raw_resort_elevation.csv')
resort_lifts = pd.read_csv('web-scraping/data/raw_resort_lifts.csv')
resort_locations = pd.read_csv('web-scraping/data/raw_resort_locations.csv')
resort_snowfall = pd.read_csv('web-scraping/data/raw_resort_snowfall.csv')
resort_terrain = pd.read_csv('web-scraping/data/raw_resort_terrain.csv')

In [5]:
# Create Master table
master = pass_info.merge(resort_lifts, on='short-name', how='left')
master = master.merge(resort_locations, on='short-name', how='left')
master = master.merge(resort_snowfall, on='short-name', how='left')
master = master.merge(resort_terrain, on='short-name', how='left')
master = master.merge(resort_elevation, on='short-name', how='left')

In [6]:
master

,pass,resort,link,short-name,High Speed Quads,Triple Chairs,Double Chairs,Gondolas &amp; Trams,High Speed Sixes,Quad Chairs,...,Skiable Terrain,Snow Making,Beginners Runs,Intermediate Runs,Advanced Runs,Expert Runs,Night Skiing,Summit,Vertical Drop,Base
0,mountain-collective,Alta Ski Area,https://www.onthesnow.com/utah/alta-ski-area/s...,alta-ski-area,3.0,1.0,2.0,NaN,NaN,NaN,...,2614 ac,140 ac,NaN,NaN,NaN,NaN,NaN,11068',2538',8530'
1,mountain-collective,Aspen Snowmass,https://www.onthesnow.com/colorado/aspen-snowm...,aspen-snowmass,15.0,3.0,5.0,3.0,1.0,4.0,...,5517 ac,658 ac,NaN,NaN,NaN,NaN,NaN,12510',4406',8104'
2,mountain-collective,Big Sky,https://www.onthesnow.com/montana/big-sky-reso...,big-sky-resort,5.0,7.0,5.0,1.0,2.0,3.0,...,5800 ac,400 ac,15%,25%,42%,18%,NaN,11166',4350',7500'
3,mountain-collective,Coronet Peak,https://www.onthesnow.com/new-zealand/coronet-...,coronet-peak,2.0,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,17%,50%,26%,21%,NaN,5410',1516',3894'
4,mountain-collective,Jackson Hole,https://www.onthesnow.com/wyoming/jackson-hole...,jackson-hole,5.0,NaN,1.0,3.0,NaN,4.0,...,2500 ac,195 ac,4%,41%,38%,17%,NaN,10450',4139',6311'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,ikon-pass,The Summit at Snoqualmie,https://www.onthesnow.com/washington/the-summi...,the-summit-at-snoqualmie,2.0,3.0,6.0,NaN,NaN,4.0,...,1994 ac,5 ac,15%,40%,45%,NaN,541 ac,3865',1025',2840'
107,ikon-pass,Thredbo Alpine Resort,https://www.onthesnow.com/australia/thredbo-al...,thredbo-alpine-resort,4.0,NaN,2.0,NaN,NaN,NaN,...,NaN,72 ac,NaN,NaN,NaN,NaN,15 ac,6683',2205',4478'
108,ikon-pass,Tremblant,https://www.onthesnow.com/quebec/tremblant/ski...,tremblant,6.0,1.0,NaN,3.0,NaN,1.0,...,754 ac,465 ac,19%,30%,37%,13%,NaN,2871',2116',755'
109,ikon-pass,Windham Mountain,https://www.onthesnow.com/new-york/windham-mou...,windham-mountain,2.0,2.0,1.0,NaN,1.0,NaN,...,285 ac,280 ac,20%,48%,19%,13%,45 ac,3100',1600',1500'


# Data Transformation

### Master Table

In [7]:
# Replace special characters in column name
master = master.rename(columns={'Gondolas &amp; Trams': 'Gondolas & Trams'})

In [8]:
# Create total chairs column
master['Total Lifts'] = master.iloc[:,4:11].sum(axis=1)

In [9]:
# Replace rowname where special characters are present
master.loc[master['short-name'] == 'folgarida-marilleva', 'resort'] = 'Folgarida Marilleva'

### Terrain

In [10]:
# Strip symbols
resort_terrain['Beginners Runs'] = resort_terrain['Beginners Runs'].str.strip('%')
resort_terrain['Intermediate Runs'] = resort_terrain['Intermediate Runs'].str.strip('%')
resort_terrain['Advanced Runs'] = resort_terrain['Advanced Runs'].str.strip('%')
resort_terrain['Expert Runs'] = resort_terrain['Expert Runs'].str.strip('%')

In [11]:
# Melt dataframe
terrain_difficulty = resort_terrain[['short-name', 'Beginners Runs', 'Intermediate Runs', 'Advanced Runs', 'Expert Runs']].melt(id_vars='short-name', var_name='run_type', value_name='Percentage')

In [12]:
# Join pass information
terrain_difficulty = terrain_difficulty.merge(pass_info, on='short-name', how='left')

### Snowfall

In [13]:
# Define function for returning snowfall in cm
def convert_to_cm(x):
    if '"' in x:
        unit = 'inch'
        value = x.replace('"', '')
        value = value.strip()
        amt = float(value) * 2.54
        return amt
    elif 'cm' in x:
        unit = 'cm'
        value = x.replace('cm', '')
        value = value.strip()
        amt = float(value)
        return amt

In [14]:
# Replace NaN with 0
resort_snowfall = resort_snowfall.fillna('0"')

In [15]:
# Apply function over month columns
for row in resort_snowfall.iterrows():
    for col in resort_snowfall.columns[1:]:
        resort_snowfall.loc[row[0], col] = convert_to_cm(resort_snowfall.loc[row[0], col])

In [16]:
# Melt dataframe
resort_snowfall = resort_snowfall.melt(id_vars='short-name', var_name='month', value_name='snowfall_cm').sort_values(by='snowfall_cm', ascending=False)

In [17]:
# Join pass information
resort_snowfall = resort_snowfall.merge(pass_info, on='short-name', how='left')

In [18]:
resort_snowfall

,short-name,month,snowfall_cm,pass,resort,link
0,jackson-hole,Feb,284.48,mountain-collective,Jackson Hole,https://www.onthesnow.com/wyoming/jackson-hole...
1,jackson-hole,Feb,284.48,ikon-pass,Jackson Hole,https://www.onthesnow.com/wyoming/jackson-hole...
2,crystal-mountain-wa,Dec,284.48,ikon-pass,Crystal Mountain,https://www.onthesnow.com/washington/crystal-m...
3,jackson-hole,Dec,271.78,mountain-collective,Jackson Hole,https://www.onthesnow.com/wyoming/jackson-hole...
4,jackson-hole,Dec,271.78,ikon-pass,Jackson Hole,https://www.onthesnow.com/wyoming/jackson-hole...
...,...,...,...,...,...,...
1327,the-summit-at-snoqualmie,Jun,0.0,ikon-pass,The Summit at Snoqualmie,https://www.onthesnow.com/washington/the-summi...
1328,tremblant,Jun,0.0,ikon-pass,Tremblant,https://www.onthesnow.com/quebec/tremblant/ski...
1329,windham-mountain,Jun,0.0,ikon-pass,Windham Mountain,https://www.onthesnow.com/new-york/windham-mou...
1330,alta-ski-area,May,0.0,mountain-collective,Alta Ski Area,https://www.onthesnow.com/utah/alta-ski-area/s...


# Export

In [19]:
# Export Master
master.to_csv('web-scraping/data/master.csv', index = False)

# Export terrain difficulty
terrain_difficulty.to_csv('web-scraping/data/terrain_difficulty.csv', index = False)

# Export terrain difficulty
resort_snowfall.to_csv('web-scraping/data/clean_resort_snowfall.csv', index = False)

In [28]:
for resort in master['resort'].drop_duplicates().sort_values():
    print(f'"{resort}",')

"Afton Alps",
"Alpe Cimbra - Folgaria - Lavarone",
"Alta Ski Area",
"Andalo - Fai della Paganella",
"Arapahoe Basin Ski Area",
"Aspen Snowmass",
"Bear Mountain",
"Beaver Creek",
"Big Sky",
"Blue Mountain",
"Boyne Mountain Resort",
"Breckenridge",
"Brides les Bains",
"Brighton Resort",
"Copper Mountain",
"Coronet Peak",
"Courchevel",
"Crested Butte Mountain Resort",
"Crystal Mountain",
"Cypress Mountain",
"Deer Valley Resort",
"Eldora Mountain Resort",
"Falls Creek Alpine Resort",
"Fernie Alpine",
"Folgarida Marilleva",
"Heavenly",
"Jackson Hole",
"June Mountain",
"Keystone",
"Kicking Horse",
"Killington Resort",
"Kimberley",
"Kirkwood",
"La Tania",
"Lake Louise",
"Les Menuires",
"Loon Mountain",
"Madonna di Campiglio",
"Mammoth Mountain",
"Meribel",
"Mont Sainte Anne",
"Monte Bondone",
"Mount Sunapee",
"Mt. Bachelor",
"Mt. Brighton",
"Mt. Buller",
"Mt. Hotham",
"Mt. Norquay",
"Nakiska Ski Area",
"Nendaz",
"Northstar California",
"Okemo Mountain Resort",
"Orelle",
"Palisades Tahoe",
"Pa